In [ ]:
import os
import pdb
import sys
import numpy as np
import argparse
import pandas as pd
from collections import Counter

In [ ]:
def tokenize(sentence, grams):
    words = sentence.split()
    tokens = []
    for gram in grams:
        for i in range(len(words) - gram + 1):
            tokens += ["_*_".join(words[i:i+gram])]
    return tokens

In [ ]:
def csv_text(df, pos_txt, neg_txt):
    for i in range(df.shape[0]):
        if(df.loc[i,'label'] == 0):    
            with open(neg_txt, 'a') as file:
                file.writelines(df.loc[i,'txt'] + "\n")

        if(df.loc[i,'label'] == 1):   
            with open(pos_txt, 'a') as file:
                file.writelines(df.loc[i,'txt'] + "\n")

    
    return pos_txt, neg_txt

In [ ]:
def build_dict(f, grams):
    dic = Counter()
    i = 0
    for sentence in open(f).readlines():
        dic.update(tokenize(sentence, grams))
        if(i < 30):
            print(tokenize(sentence, grams))
            i = i + 1
    return dic

In [ ]:
def process_files(file_pos, file_neg, dic, r, outfn, grams):
    output = []
    for beg_line, f in zip(["1", "-1"], [file_pos, file_neg]):
        for l in open(f).readlines():
            tokens = tokenize(l, grams)
            indexes = []
            for t in tokens:
                try:
                    indexes += [dic[t]]
                except KeyError:
                    pass
            indexes = list(set(indexes))
            indexes.sort()
            line = [beg_line]
            for i in indexes:
                line += ["%i:%f" % (i + 1, r[i])]
            output += [" ".join(line)]
    output = "\n".join(output)
    f = open(outfn, "w")
    f.writelines(output)
    f.close()

In [ ]:
def compute_ratio(poscounts, negcounts, alpha=1):
    alltokens = list(set(list(poscounts.keys()) + list(negcounts.keys())))
    dic = dict((t, i) for i, t in enumerate(alltokens))
    d = len(dic)
    print("computing r...",d)
    p, q = np.ones(d) * alpha , np.ones(d) * alpha
    i = 0
    for t in alltokens:
        p[dic[t]] += poscounts[t]
        q[dic[t]] += negcounts[t]
        p /= abs(p).sum()
        q /= abs(q).sum()
        rt = np.log(p/q)
        if(i < 30):
            print(t, max(p), p)
            print(t,q, max(q))
        i = i + 1
        
    p /= abs(p).sum()
    q /= abs(q).sum()
    r = np.log(p/q)
    return dic, r

In [ ]:
df = pd.read_csv("data.csv", error_bad_lines=False)
csv_text(df,"test-true.txt", "test-falseg.txt")

In [ ]:
ptrain = "train-true.txt"
ntrain = "train-false.txt"
ptest = "test-true.txt"
ntest = "test-false.txt"
liblinear = "liblinear-1.96"
out = "TEST-SCORE"
ngram = "2"
ngram = [int(i) for i in ngram]
print("counting...")

poscounts = build_dict(ptrain, ngram)         
negcounts = build_dict(ntrain, ngram)         

dic, r = compute_ratio(poscounts, negcounts)
print("processing files...")
process_files(ptrain, ntrain, dic, r, "train-nbsvm.txt", ngram)
process_files(ptest, ntest, dic, r, "test-nbsvm.txt", ngram)

print(r, dict)
trainsvm = os.path.join(liblinear, "train") 
predictsvm = os.path.join(liblinear, "predict") 
os.system(trainsvm + " -s 0 train-nbsvm.txt model.logreg")
os.system(predictsvm + " -b 1 test-nbsvm.txt model.logreg " + out)
os.system("rm model.logreg train-nbsvm.txt test-nbsvm.txt")
print("complete")